In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition, model_selection
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [ ]:
data_dir = 'input_data'
seed=0

# Reading in training dataset
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'), index_col='ID')

## PCA Reduction

First, comparing performance of various PCA kernels

In [ ]:
# Scaling features to means
std_scale = preprocessing.StandardScaler().fit(train_df.iloc[:, 1:])
train_df_scaled = std_scale.transform(train_df.iloc[:, 1:])

In [ ]:
# Applying Box-Cox transform to target
train_df['target'] = stats.boxcox(train_df['target'])[0]

In [ ]:
# Looping through kernels and plotting component correlations to target
kernels=['linear', 'poly', 'rbf', 'sigmoid', 'cosine']
n_components=200

fig = plt.figure(figsize=(20, 20))

kernel_outputs = {}

for i, kernel in enumerate(kernels):
    kpca_sm = decomposition.KernelPCA(n_components=n_components, kernel=kernel)
    kpca_sm.fit(train_df_scaled)
    train_df_kpca = pd.concat([train_df['target'], pd.DataFrame(kpca_sm.transform(train_df_scaled), columns=['c{}'.format(num+1) for num in range(n_components)], index=train_df.index)], axis=1)
    correlation_kpca = train_df_kpca.corr(method='spearman')['target'][1:]
    
    #Saving results in dictionary for post-evaluation use
    kernel_outputs[kernel] = {}
    kernel_outputs[kernel]['coeffs'] = correlation_kpca
    kernel_outputs[kernel]['data']= train_df_kpca
    kernel_outputs[kernel]['transformer'] = kpca_sm
    
    #Plotting results
    ax = fig.add_subplot(3, 2, i+1)
    ax.scatter(range(len(correlation_kpca)), np.sort(correlation_kpca))
    ax.set_title('Component/Target Correlation ({} Kernel)'.format(kernel))
    ax.set_xlabel('Components')
    ax.set_ylabel('Correlation')
    ax.set_ylim(-.3, .3)
    

The polynomial kernel appears to have the highest magnitude of correlations to the target. Before selecting an option, checking the performance of a sparse PCA method.

In [ ]:
#Instantiating and fitting sparse PCA algorithm
sparse_sm = decomposition.SparsePCA(n_components=n_components)
sparse_sm.fit(train_df_scaled)
train_df_sparse = pd.concat([train_df['target'], pd.DataFrame(sparse_sm.transform(train_df.iloc[:, 1:].values), columns=['c{}'.format(num+1) for num in range(n_components)], index=train_df.index)], axis=1)
correlation_sparse = train_df_sparse.corr(method='spearman')['target'][1:]

#Saving outputs to earlier created dictionary
kernel_outputs['sparse'] = {}
kernel_outputs['sparse']['coeffs'] = correlation_sparse
kernel_outputs['sparse']['data'] = train_df_sparse
kernel_outputs['sparse']['transformer'] = sparse_sm

#Plotting output correlation
plt.scatter(range(len(correlation_sparse)), np.sort(correlation_sparse))

## Choosing and Applying a PCA Model

PCA using a polynomial model appeared to perform the best. Using a gridsearch to optimize the paramaters, assuming this will be ultimately fitted to a gradient boosting regression model. Selecting components meeting correlation threshold and saving to a dataframe for regression modeling.

In [ ]:
# Optimizing parameters

poly_kpca = decomposition.KernelPCA(n_components=n_components, kernel='poly')

GBRegressor = GradientBoostingRegressor()

X = train_df_scaled
y = train_df['target'] 

pipe = Pipeline(steps=[('pca', poly_kpca), ('grb_regressor', GBRegressor)])

param_grid = {
    'pca__gamma': [1, 10, 100, 1000],
    'pca__degree': [3, 4, 5],
}

pca_search = GridSearchCV(pipe, param_grid, n_jobs=-1)
pca_search.fit(X, y)

pca_search.best_params_

In [ ]:
optimized_kpca = pca_search.best_estimator_[0]

In [ ]:
# Fitting PCA algorithm to data

optimized_kpca.fit(train_df_scaled)
train_df_optimized = pd.concat([train_df['target'], pd.DataFrame(optimized_kpca.transform(train_df_scaled), columns=['c{}'.format(num+1) for num in range(n_components)], index=train_df.index)], axis=1)
correlation_optimized = train_df_optimized.corr(method='spearman')['target'][1:]

In [ ]:
# Reducing components based on correlation threshold

pca_threshold = .1

best_components = correlation_optimized[abs(correlation_optimized) >= pca_threshold].sort_values(ascending=False)
pca_cols = [['target'] + list(best_components.index)][0]
train_df_final_pca = train_df_optimized.loc[:, pca_cols]

In [ ]:
#Saving component-populated dataframe to local drive
train_df_final_pca.to_csv(os.path.join(data_dir, 'train_pca.csv'))

In [ ]:
# Freeing memory
train_df_final_pca = None
train_df = None

Transforming test data using fitted standard scaler and PCA model

In [ ]:
#Reading in test data
chunksize = 5000
test_df_pca = pd.DataFrame()

for chunk in pd.read_csv(os.path.join(data_dir, 'test.csv'), index_col='ID', chunksize=chunksize):
    test_df_pca = pd.concat([test_df_pca, chunk])

In [ ]:
#Transforming test data using selected PCA kernel
test_df_pca = pd.DataFrame(optimized_kpca.transform(std_scale.transform(test_df_pca.values)), columns=['c{}'.format(num+1) for num in range(n_components)], index=test_df_pca.index).loc[:, list(best_components.index)]

In [ ]:
test_df_pca.to_csv(os.path.join(data_dir, 'test_pca.csv'))